In [1]:
import random as r
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.special import gamma
from scipy.special import beta
from scipy.integrate import quad
from scipy.stats import moment

In [93]:
class linear_reg:
    def __init__(self, sus_matrix_factor, reaction):
        self.factor = np.array([[1, *list(sus_matrix_factor[i])] for i in range(len(sus_matrix_factor))])
        self.reaction = reaction
        self.m_fisher = np.transpose(self.factor).dot(self.factor)
        self.betta = np.linalg.inv(self.m_fisher).dot(np.transpose(self.factor)).dot(self.reaction)
        self.e = self.reaction - self.factor.dot(self.betta)
        self.RSS = np.transpose(self.e).dot(self.e)
        self.TSS = np.sum((self.reaction - self.reaction.mean())**2)
        self.R2 = (self.TSS - self.RSS) / self.TSS
        
    def predict(self, x):
        sum = self.betta[0]
        for i in range(1, self.betta.shape[0]):
            sum += self.betta[i] * x[i - 1]
        return sum

In [3]:
def rand_strange_dense(x1,x2,x3,x4,x5):
    return np.random.normal(2+3*x1-2*x2+x3+x4-x5, 1.5)

In [16]:
fisher_dense = (lambda x, d1, d2: np.sqrt((d1*x)**d1 * d2**d2 / (d1*x + d2)**(d1 + d2)) / x / beta(d1/2, d2/2))

In [29]:
gaus = lambda x, m, sigma: 1 / np.sqrt(2*np.pi) / sigma * np.exp(-(x-m)**2/2/sigma**2)

In [23]:
stud_dense = lambda x, n: gamma(0.5 * (n + 1)) / gamma(0.5 * n) / np.sqrt(n*np.pi) / (1 + x**2 / n)**(0.5 * (n + 1))

In [251]:
alpha = 0.05

# T1

In [261]:
# генерация векторов

In [4]:
ksi_s = [list(np.random.sample(5) * 2 - 1) for i in range(50)]

In [5]:
react = np.array([rand_strange_dense(*ksi_s[i]) for i in range(len(ksi_s))])

In [269]:
reg = linear_reg(ksi_s, react)

## a)

In [223]:
R2_ksi_s = list()
for i in range(5):
    ksi_i = np.array([ksi_s[j][i] for j in range(50)])
    mod_ksi_s = [[*ksi_s[j][0:i], *ksi_s[j][i+1:5]] for j in range(50)]
    ksi_reg_i = linear_reg(mod_ksi_s, ksi_i)
    R2_ksi_s.append(ksi_reg_i.R2)
print(*[f"R2_ksi_{i+1} = {ksi_val}\n" for i, ksi_val in enumerate(R2_ksi_s)])
if (np.where(np.array(R2_ksi_s) >= 0.7)[0].shape[0] > 0):
    print("есть мультиколлинеарность")
else:
    print("мультиколлинеарности нет")

R2_ksi_1 = 0.1095389960371652
 R2_ksi_2 = 0.12043120088382755
 R2_ksi_3 = 0.0181849246127577
 R2_ksi_4 = 0.09019916668160922
 R2_ksi_5 = 0.0928784501484061

мультиколлинеарности нет


## b)

In [295]:
print(f"eta =", f"{round(reg.betta[0], 3)}", *[(f"+" if reg.betta[i] > 0 else f"-") + f" {abs(round(reg.betta[i], 3))}*ksi{i}" for i in range(1, 6)])

eta = 2.042 + 3.233*ksi1 - 2.17*ksi2 + 1.478*ksi3 + 0.613*ksi4 - 0.822*ksi5


In [260]:
start_values = [abs(reg.betta[i] * np.sqrt(44) / np.sqrt(reg.RSS * np.linalg.inv(reg.m_fisher)[i][i])) for i in range(6)]
p_val_beta = list()
for i in range(6):
    p_val_beta.append(2*quad((lambda x: stud_dense(x, 44)), start_values[i], np.inf)[0])

print(*[f"beta{i} " + ("значим" if p_val_beta[i] < alpha else "незначим") + f", p-val = {round(p_val_beta[i],3)}\n" for i in range(6)])

beta0 значим, p-val = 0.0
 beta1 значим, p-val = 0.0
 beta2 значим, p-val = 0.0
 beta3 значим, p-val = 0.0
 beta4 незначим, p-val = 0.068
 beta5 значим, p-val = 0.015



## c)

In [262]:
print("коэффициент детерминации = ", round(reg.R2, 3))

коэффициент детерминации =  0.829


In [281]:
start_val = reg.R2 / (1 - reg.R2) * 44 / 5
p_val_of_determination = quad((lambda x: fisher_dense(x, 5, 44)), start_val, np.inf)[0]
if p_val_of_determination < alpha:
    print("коэффициент детерминации значим, p-val = ", p_val_of_determination)
else:
    print("коэффициент детерминации незначим")

коэффициент детерминации значим, p-val =  8.8294155804958265e-16


## d)

In [19]:
b = 0.95

In [288]:
vec = np.array([1,0,0,0,0,0])
val_in_zeros = reg.predict(vec[1:])
A = 1 + vec.dot(np.linalg.inv(reg.m_fisher)).dot(np.transpose(vec))

In [24]:
def find_left_edge(t_b):
    left = -1
    right = 0
    while 2*quad((lambda x: stud_dense(x, 44)), -np.inf, left)[0] > t_b:
        left *= 2

    epsilon = 0.0001
    while True:
        integral = 2*quad((lambda x: stud_dense(x, 44)), -np.inf, 0.5 * (left + right))[0]
        if integral > t_b:
            right = 0.5 * (left + right)
        else:
            left = 0.5 * (left + right)
            
        if abs(integral - t_b) < epsilon:
            break
            
    return 0.5 * (left + right)

In [277]:
left_edge = find_left_edge((1 - b) / 2)
right_edge = abs(left_edge)
interval = [(val_in_zeros - right_edge) * np.sqrt(A*reg.RSS/44), (val_in_zeros - left_edge) * np.sqrt(A*reg.RSS/44)]
print("значение в точки 0 = ", val_in_zeros)
print(f"доверительный интервал = ({round(interval[0], 3)}, {round(interval[1], 3)})")

значение в точки 0 =  2.041957070453671
доверительный интервал = (-0.342, 5.288)


## e)

In [282]:
I = 0
for i in range(reg.e.shape[0]):
    for j in range(i+1, reg.e.shape[0]):
        if reg.e[i] > reg.e[j]:
            I += 1
vval = (I - 50*49/4) / np.sqrt(50**3/36)
er_p_val = 2 * quad((lambda x: gaus(x, 0, 1)), abs(vval), np.inf)[0]
if (er_p_val >= alpha):
    print("ошибки измерения независимы")
else:
    print("ошибки измерения зависимы, p-val = ", er_p_val)

ошибки измерения независимы


## f)

In [38]:
N = 50000
e = reg.e.copy()
e.sort()
er_m = reg.e.mean()
er_sigma = np.sqrt(moment(e, 2))
delta = np.sqrt(50)*max([abs(quad(lambda x: gaus(x, er_m, er_sigma), -np.inf, el)[0] - (np.where(e < el)[0][-1] + 1)/50) for el in e[1:]] + [abs(quad(lambda x: gaus(x, er_m, er_sigma), -np.inf, el)[0] - (np.where(e <= el)[0][-1]+1)/50) for el in e])
bootstrap_deltas = list()
for i in range(N):
    new_es = np.random.normal(er_m, er_sigma, 50)
    new_er_m = new_es.mean()
    new_er_sigma = np.sqrt(moment(reg.e, 2))
    new_es.sort()
    bootstrap_deltas.append(np.sqrt(50)*max([abs(quad(lambda x: gaus(x, new_er_m, new_er_sigma), -np.inf, el)[0] - (np.where(new_es < el)[0][-1] + 1)/50) for el in new_es[1:]] + [abs(quad(lambda x: gaus(x, new_er_m, new_er_sigma), -np.inf, el)[0] - (np.where(new_es <= el)[0][-1]+1)/50) for el in new_es]))
    
bootstrap_deltas = np.array(bootstrap_deltas)
bootstrap_deltas.sort()
k = np.where(bootstrap_deltas < delta)[0][-1]
er_norm_p_val = 1 - (k)/N

In [283]:
if er_norm_p_val >= alpha:
    print("распределение ошибок нормально")
else:
    print("распределение ошибок не нормально, p-val = ", er_norm_p_val)

распределение ошибок нормально


## g)

In [284]:
errors = abs(reg.e.copy())
doubled_err_sigma = 2*np.median(errors) / 0.675
for i in range(50):
    if (errors[i] >= doubled_err_sigma):
        print(i, " - выброс")

36  - выброс
39  - выброс


## h)

In [286]:
CVSS = 0
for i in range(50):
    cross_ksi_s_i = ksi_s[:i] + ksi_s[i + 1:]
    cross_react = np.array(list(react[:i]) + list(react[i + 1:]))
    control_ksi_i = ksi_s[i]
    control_react = react[i]
    cross_reg = linear_reg(cross_ksi_s_i, cross_react)
    CVSS += (control_react - cross_reg.predict(control_ksi_i))**2
R2_CV = (reg.TSS - CVSS) / reg.TSS
print("R2_CV = ", R2_CV)

R2_CV =  0.7851519433353783


## i)

In [287]:
# возьмём точку x = [-0.5,0.5,-0.5,0.5,-0.5]
test_x = [-0.5, 0.5, -0.5, 0.5, -0.5]
test_y = [rand_strange_dense(*test_x) for i in range(5)]
strange_sigma = moment(test_y, 2)
delta_v = reg.RSS / 44 / strange_sigma**2
p_val = quad((lambda x: fisher_dense(x, 44, 4)), delta_v, np.inf)[0]

if p_val >= alpha:
    print("модель адекватна")
else:
    print("модель не адекватна")

модель адекватна


## j)

In [302]:
del_ind = p_val_beta.index(max(p_val_beta))
print(F"удаляем {del_ind+1} коэффициент")
new_ksi_s = [[*ksi_s[i][:del_ind - 1], *ksi_s[i][del_ind:]] for i in range(50)]
new_j_reg = linear_reg(new_ksi_s, react)
print(f"eta =", f"{round(new_j_reg.betta[0], 3)}", *[(f"+" if new_j_reg.betta[i] > 0 else f"-") + f" {abs(round(new_j_reg.betta[i], 3))}*ksi{i}" for i in range(1, 5)])

удаляем 5 коэффициент
eta = 2.058 + 3.073*ksi1 - 2.231*ksi2 + 1.519*ksi3 - 0.77*ksi4


In [297]:
new_j_start_values = [abs(new_j_reg.betta[i] * np.sqrt(45) / np.sqrt(new_j_reg.RSS * np.linalg.inv(new_j_reg.m_fisher)[i][i])) for i in range(5)]
new_j_p_val_beta = list()
for i in range(5):
    new_j_p_val_beta.append(2*quad((lambda x: stud_dense(x, 45)), new_j_start_values[i], np.inf)[0])

print(*[f"beta{i} " + ("значим" if new_j_p_val_beta[i] < alpha else "незначим") + f", p-val = {round(new_j_p_val_beta[i],3)}\n" for i in range(5)])

beta0 значим, p-val = 0.0
 beta1 значим, p-val = 0.0
 beta2 значим, p-val = 0.0
 beta3 значим, p-val = 0.0
 beta4 значим, p-val = 0.024



In [298]:
print("коэффициент детерминации = ", round(new_j_reg.R2, 3))

коэффициент детерминации =  0.815


In [299]:
new_j_start_val = new_j_reg.R2 / (1 - new_j_reg.R2) * 45 / 4
new_j_p_val_of_determination = quad((lambda x: fisher_dense(x, 4, 45)), new_j_start_val, np.inf)[0]

if new_j_p_val_of_determination < alpha:
    print("коэффициент детерминации значим, p-val = ", new_j_p_val_of_determination)
else:
    print("коэффициент детерминации незначим")

коэффициент детерминации значим, p-val =  6.1411825165783155e-16


In [301]:
comp_delta = (new_j_reg.RSS - reg.RSS) / reg.RSS * (50 - 6) / (6 - 5)
comp_p_val = quad((lambda x: fisher_dense(x, 6 - 5, 50 - 6)), comp_delta, np.inf)[0]

if comp_p_val >= alpha:
    print("нет оснований отвергнуть гипотезу о том, что усложнение модели регрессии не является значимым")
else:
    print("отвергаем гипотезу о том, что усложнение модели регрессии не является значимым, p-val = ", comp_p_val)

нет оснований отвергнуть гипотезу о том, что усложнение модели регрессии не является значимым


## k)

In [310]:
comp_deltas = list()
for i in range(1000):
    i_i = np.random.randint(0, 50, 50)
    comp_ksi_s = [ksi_s[j] for j in i_i]
    comp_react = np.array([react[j] for j in i_i])
    comp_old_reg = linear_reg(comp_ksi_s, comp_react)
    
    new_j_i_i = np.random.randint(0, 50, 50)
    comp_new_j_ksi_s = [new_ksi_s[j] for j in new_j_i_i]
    comp_new_j_react = np.array([react[j] for j in new_j_i_i])
    comp_new_reg = linear_reg(comp_new_j_ksi_s, comp_new_j_react)
    
    comp_deltas.append((comp_new_reg.RSS - comp_old_reg.RSS) / comp_old_reg.RSS * (50 - 6) / (6 - 5))
    
comp_deltas = np.array(comp_deltas)
comp_deltas.sort()
k = np.where(comp_deltas < comp_delta)[0][-1]
b_comp_p_val = 1 - (k)/N

if b_comp_p_val >= alpha:
    print("нет оснований отвергнуть гипотезу о том, что усложнение модели регрессии не является значимым")
else:
    print("отвергаем гипотезу о том, что усложнение модели регрессии не является значимым, p-val = ", b_comp_p_val)

нет оснований отвергнуть гипотезу о том, что усложнение модели регрессии не является значимым


# T2

In [357]:
class linear_reg2:
    def __init__(self, sus_matrix_factor, reaction):
        self.factor = np.array(sus_matrix_factor)
        self.reaction = reaction
        self.m_fisher = np.transpose(self.factor).dot(self.factor)
        self.betta = np.linalg.inv(self.m_fisher).dot(np.transpose(self.factor)).dot(self.reaction)
        self.e = self.reaction - self.factor.dot(self.betta)
        self.RSS = np.transpose(self.e).dot(self.e)
        self.TSS = np.sum((self.reaction - self.reaction.mean())**2)
        self.R2 = (self.TSS - self.RSS) / self.TSS
        
    def predict(self, x):
        for i in range(self.betta.shape[0]):
            sum += self.betta[i] * x[i - 1]
        return sum

In [355]:
array = [[83, 85],
         [84,85,85,86,86,87],
         [86,87,87,87,88,88,88,88,88,89,90],
         [89,90,90,91],
         [90,92]]
N = sum([len(array[i]) for i in range(len(array))])

ksi_s = list()
for i in range(len(array)):
    for j in range(len(array[i])):
        a = np.zeros(5, dtype = int)
        a[i] += 1
        ksi_s.append(list(a))

react = list()
for i in range(len(array)):
    react += array[i]
react = np.array(react)

In [358]:
reg2 = linear_reg2(ksi_s, react)

In [376]:
reg2_inv_m_fisher = np.linalg.inv(reg2.m_fisher)

In [366]:
print(f"eta =", f"{round(reg2.betta[0], 3)}*ksi1", *[(f"+" if reg2.betta[i] > 0 else f"-") + f" {abs(round(reg2.betta[i], 3))}*ksi{i+1}" for i in range(1, 5)])

eta = 84.0*ksi1 + 85.5*ksi2 + 87.818*ksi3 + 90.0*ksi4 + 91.0*ksi5


## a)

In [367]:
print("коэффициент детерминации = ", round(reg2.R2, 3))

коэффициент детерминации =  0.811


In [375]:
start_val = reg2.R2 / (1 - reg2.R2) * (N - reg2.betta.shape[0]) / (reg2.betta.shape[0] - 1)
p_val_of_determination = quad((lambda x: fisher_dense(x, reg2.betta.shape[0] - 1, N - reg2.betta.shape[0])), start_val, np.inf)[0]
if p_val_of_determination < alpha:
    print("коэффициент детерминации значим, p-val = ", p_val_of_determination, "\n => возраст влияет на содержание иммуноглоублина")
else:
    print("коэффициент детерминации незначим")

коэффициент детерминации значим, p-val =  5.407410946440232e-07 
 => возраст влияет на содержание иммуноглоублина


## b)

In [380]:
for i in range(reg2.betta.shape[0]):
    for j in range(i + 1, reg2.betta.shape[0]):
        start_val_i_j = abs(np.sqrt((N - reg2.betta.shape[0]) / reg2.RSS / (reg2_inv_m_fisher[i][i] + reg2_inv_m_fisher[j][j])) * (reg2.betta[i] - reg2.betta[j]))
        p_val_i_j = 2*quad((lambda x: stud_dense(x, N - reg2.betta.shape[0])), start_val_i_j, np.inf)[0]
        if p_val_i_j < alpha:
            print(f"среднее {i + 1} нельзя считать равным среднему {j + 1}", " p-val = ", round(p_val_i_j, 5))
        else:
            print(f"среднее {i + 1} можно считать равным среднему {j + 1}")

среднее 1 можно считать равным среднему 2
среднее 1 нельзя считать равным среднему 3  p-val =  0.00017
среднее 1 нельзя считать равным среднему 4  p-val =  0.0
среднее 1 нельзя считать равным среднему 5  p-val =  0.0
среднее 2 нельзя считать равным среднему 3  p-val =  0.0004
среднее 2 нельзя считать равным среднему 4  p-val =  0.0
среднее 2 нельзя считать равным среднему 5  p-val =  0.0
среднее 3 нельзя считать равным среднему 4  p-val =  0.00239
среднее 3 нельзя считать равным среднему 5  p-val =  0.001
среднее 4 можно считать равным среднему 5
